# Imports

In [ ]:
%%capture
!pip install py2neo

In [ ]:
from py2neo import Graph

# Put your address and password here.

In [ ]:
bolt = "bolt://3.239.227.179:7687"
pswrd = "cracks-execution-towel"
graph = Graph(bolt, auth=("neo4j", pswrd))

# Import Queries

In [ ]:
import_races_query = """

// Prepare a SPARQL query 
WITH 'SELECT ?item ?itemLabel 
      WHERE{ ?item wdt:P31 wd:Q989255 . 
      SERVICE wikibase:label { bd:serviceParam wikibase:language 
      "[AUTO_LANGUAGE],en" } }' AS sparql 
// make a request to Wikidata
CALL apoc.load.jsonParams('https://query.wikidata.org/sparql?query=' + 
                           apoc.text.urlencode(sparql), 
                         { Accept: "application/sparql-results+json"}, null) 
YIELD value 
// Unwind results to row 
UNWIND value['results']['bindings'] as row 
// Prepare data 
WITH row['itemLabel']['value'] as race, 
     row['item']['value'] as url, 
     split(row['item']['value'],'/')[-1] as id 
// Store to Neo4j 
CREATE (r:Race) SET r.race = race, 
                    r.url = url, 
                    r.id = id

"""

graph.run(import_races_query);

In [ ]:
import_characters_query = """

// Iterate over each race in graph
MATCH (r:Race)
// Prepare a SparQL query
WITH 'SELECT ?item ?itemLabel 
      WHERE { ?item wdt:P31 wd:' + r.id + ' . 
      SERVICE wikibase:label { bd:serviceParam wikibase:language 
      "[AUTO_LANGUAGE],en" } }' AS sparql, r 
// make a request to Wikidata 
CALL apoc.load.jsonParams( "https://query.wikidata.org/sparql?query=" + 
                            apoc.text.urlencode(sparql), 
                            { Accept: "application/sparql-results+json"}, null)
YIELD value 
UNWIND value['results']['bindings'] as row 
WITH row['itemLabel']['value'] as name, 
     row['item']['value'] as url, 
     split(row['item']['value'],'/')[-1] as id, r 
// Store to Neo4j 
CREATE (c:Character) 
SET c.name = name, 
    c.url = url, 
    c.id = id 
CREATE (c)-[:BELONG_TO]->(r)

"""

graph.run(import_characters_query);

In [ ]:
import_gender_query = """
// Iterate over characters 
MATCH (r:Character) 
// Prepare a SparQL query 
WITH 'SELECT * 
      WHERE{ ?item rdfs:label ?name . 
              filter (?item = wd:' + r.id + ') 
              filter (lang(?name) = "en" ) . 
      OPTIONAL{ ?item wdt:P21 [rdfs:label ?gender] . 
                filter (lang(?gender)="en") } 
      OPTIONAL{ ?item wdt:P27 [rdfs:label ?country] . 
              filter (lang(?country)="en") } 
      OPTIONAL{ ?item wdt:P1196 [rdfs:label ?death] . 
              filter (lang(?death)="en") }}' AS sparql, r 
// make a request to Wikidata 
CALL apoc.load.jsonParams( "https://query.wikidata.org/sparql?query=" 
    + apoc.text.urlencode(sparql), 
    { Accept: "application/sparql-results+json"}, null)
YIELD value 
UNWIND value['results']['bindings'] as row 
SET r.gender = row['gender']['value'], 
    r.manner_of_death = row['death']['value'] 
// Execute FOREACH statement 
FOREACH(ignoreme in case when row['country'] is not null then [1] else [] end | 
    MERGE (c:Country{name:row['country']['value']}) 
    MERGE (r)-[:IN_COUNTRY]->(c))

"""

while True:
    try:
        graph.run(import_gender_query)
        print('Done.')
        break
    except:
        print('Please wait...')

In [ ]:
import_social_query = """

// Iterate over characters 
MATCH (r:Character) 
WITH 'SELECT * 
      WHERE{ ?item rdfs:label ?name . 
             filter (?item = wd:' + r.id + ') 
             filter (lang(?name) = "en" ) . 
      OPTIONAL{ ?item wdt:P22 ?father } 
      OPTIONAL{ ?item wdt:P25 ?mother } 
      OPTIONAL{ ?item wdt:P1038 ?relative } 
      OPTIONAL{ ?item wdt:P3373 ?sibling } 
      OPTIONAL{ ?item wdt:P26 ?spouse }}' AS sparql, r 
// make a request to wikidata 
CALL apoc.load.jsonParams( "https://query.wikidata.org/sparql?query=" + 
    apoc.text.urlencode(sparql), 
    { Accept: "application/sparql-results+json"}, null) YIELD value 
UNWIND value['results']['bindings'] as row 
FOREACH(ignoreme in case when row['mother'] is not null then [1] else [] end | 
    MERGE (c:Character{url:row['mother']['value']}) 
    MERGE (r)-[:HAS_MOTHER]->(c)) 
FOREACH(ignoreme in case when row['father'] is not null then [1] else [] end | 
    MERGE (c:Character{url:row['father']['value']}) 
    MERGE (r)-[:HAS_FATHER]->(c)) 
FOREACH(ignoreme in case when row['relative'] is not null then [1] else [] end | 
    MERGE (c:Character{url:row['relative']['value']}) 
    MERGE (r)-[:HAS_RELATIVE]-(c)) 
FOREACH(ignoreme in case when row['sibling'] is not null then [1] else [] end | 
    MERGE (c:Character{url:row['sibling']['value']}) 
    MERGE (r)-[:SIBLING]-(c))
FOREACH(ignoreme in case when row['spouse'] is not null then [1] else [] end | 
    MERGE (c:Character{url:row['spouse']['value']}) 
    MERGE (r)-[:SPOUSE]-(c))

"""

while True:
    try:
        graph.run(import_social_query)
        print('Done.')
        break
    except:
        print('Please wait...')

In [ ]:
import_groups_query = """

MATCH (r:Character) 
WHERE exists (r.id) 
WITH 'SELECT * 
      WHERE{ ?item rdfs:label ?name . 
             filter (?item = wd:' + r.id + ') 
             filter (lang(?name) = "en" ) . 
      OPTIONAL { ?item wdt:P106 [rdfs:label ?occupation ] . 
           filter (lang(?occupation) = "en" ). } 
      OPTIONAL { ?item wdt:P103 [rdfs:label ?language ] . 
           filter (lang(?language) = "en" ) . } 
      OPTIONAL { ?item wdt:P463 [rdfs:label ?member_of ] . 
           filter (lang(?member_of) = "en" ). } 
      OPTIONAL { ?item wdt:P1344[rdfs:label ?participant ] . 
           filter (lang(?participant) = "en") . } 
      OPTIONAL { ?item wdt:P39[rdfs:label ?position ] . 
           filter (lang(?position) = "en") . }}' AS sparql, r 
CALL apoc.load.jsonParams( "https://query.wikidata.org/sparql?query=" + 
                             apoc.text.urlencode(sparql), 
                             { Accept: "application/sparql-results+json"}, null) 
YIELD value 
UNWIND value['results']['bindings'] as row 
FOREACH(ignoreme in case when row['language'] is not null then [1] else [] end | 
        MERGE (c:Language{name:row['language']['value']}) 
        MERGE (r)-[:HAS_LANGUAGE]->(c)) 
FOREACH(ignoreme in case when row['occupation'] is not null then [1] else [] end | 
        MERGE (c:Occupation{name:row['occupation']['value']}) 
        MERGE (r)-[:HAS_OCCUPATION]->(c)) 
FOREACH(ignoreme in case when row['member_of'] is not null then [1] else [] end | 
        MERGE (c:Group{name:row['member_of']['value']}) 
        MERGE (r)-[:MEMBER_OF]->(c)) 
FOREACH(ignoreme in case when row['participant'] is not null then [1] else [] end | 
        MERGE (c:Event{name:row['participant']['value']}) 
        MERGE (r)-[:PARTICIPATED]->(c)) 
SET r.position = row['position']['value']

"""

while True:
    try:
        graph.run(import_groups_query)
        print('Done.')
        break
    except:
        print('Please wait...')

In [ ]:
import_enemy_query = """

MATCH (r:Character) 
WHERE exists (r.id) 
WITH 'SELECT * 
      WHERE { ?item rdfs:label ?name . 
              filter (?item = wd:' + r.id + ') 
              filter (lang(?name) = "en" ) . 
      OPTIONAL{ ?item wdt:P1830 [rdfs:label ?owner ] . 
            filter (lang(?owner) = "en" ). } 
      OPTIONAL{ ?item wdt:P7047 ?enemy }}' AS sparql, r 
CALL apoc.load.jsonParams( "https://query.wikidata.org/sparql?query=" + 
                            apoc.text.urlencode(sparql), 
                            { Accept: "application/sparql-results+json"}, null) 
YIELD value 
WITH value,r 
WHERE value['results']['bindings'] <> [] 
UNWIND value['results']['bindings'] as row 
FOREACH(ignoreme in case when row['owner'] is not null then [1] else [] end |
    MERGE (c:Item{name:row['owner']['value']}) 
    MERGE (r)-[:OWNS_ITEM]->(c)) 
FOREACH(ignoreme in case when row['enemy'] is not null then [1] else [] end | 
    MERGE (c:Character{url:row['enemy']['value']}) 
    MERGE (r)-[:ENEMY]->(c))

"""

while True:
    try:
        graph.run(import_enemy_query)
        print('Done')
        break
    except:
        print('Please wait...')